In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns 

In [2]:
data=pd.read_csv("database.csv.zip",low_memory=False)

In [3]:
data= data[data["Perpetrator Age"].str.contains(' ') == False]
data["Perpetrator Age"] = data["Perpetrator Age"].astype(int)

# Preprocessing

In [4]:
idx_to_drop=data[data["Crime Solved"]=="No"].index
data.drop(idx_to_drop,inplace=True)

In [5]:
data.drop(["Record ID","Agency Code","Agency Name","Agency Type","Record Source"],axis=1,inplace=True)

In [6]:
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()

In [7]:
data.loc[:,"Perpetrator Sex"]=data.loc[:,"Perpetrator Sex"].map({"Male":int(1),"Female":int(2),"Unknown":int(0)})

In [8]:
data.loc[:,"Victim Sex"]=data.loc[:,"Victim Sex"].map({"Male":int(1),"Female":int(2),"Unknown":int(0)})

In [9]:
data.loc[:,"Perpetrator Race"]=data.loc[:,"Perpetrator Race"].map({'Native American/Alaska Native':int(1),'White':int(2),'Black':int(3),'Asian/Pacific Islander':int(4),'Unknown':int(0)})

In [10]:
data.loc[:,"Victim Race"]=data.loc[:,"Victim Race"].map({'Native American/Alaska Native':int(1),'White':int(2),'Black':int(3),'Asian/Pacific Islander':int(4),'Unknown':int(0)})

In [11]:
data.loc[:,"Perpetrator Ethnicity"]=data.loc[:,"Perpetrator Ethnicity"].map({"Unknown":int(0),"Not Hispanic":int(1),"Hispanic":int(2)})

In [12]:
data.loc[:,"Victim Ethnicity"]=data.loc[:,"Victim Ethnicity"].map({"Unknown":int(0),"Not Hispanic":int(1),"Hispanic":int(2)})

In [13]:
data.loc[:,"Relationship"]=(encoder.fit_transform(data.loc[:,"Relationship"])).astype("int64")

In [14]:
data.loc[:,"Weapon"]=(encoder.fit_transform(data.loc[:,"Weapon"])).astype("int64")

In [15]:
data.loc[:,"City"]=(encoder.fit_transform(data.loc[:,"City"])).astype("int64")

In [16]:
data.loc[:,"State"]=(encoder.fit_transform(data.loc[:,"State"])).astype("int64")

In [17]:
data.loc[:,"Month"]=data.loc[:,"Month"].map(({'January':int(1),"February":int(2),"March":int(3),'April':int(4),'May':int(5),'June':int(6),'July':int(7),'August':int(8),'September':int(9),'October':int(10),'November':int(11),'December':int(12)})).astype("int64")

In [18]:
data.loc[:,"Crime Solved"]=data.loc[:,"Crime Solved"].map({"Yes":int(1),"No":int(0)})

In [19]:
data.loc[:,"Crime Type"]=data.loc[:,"Crime Type"].map({'Murder or Manslaughter':int(1),'Manslaughter by Negligence':int(2)})

# Supervised ML part2 

In [20]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn.linear_model import LogisticRegression 
from sklearn.linear_model import LinearRegression 
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics

#### Predicting perpetrator's Age

In [22]:
newdata=data[(data["Perpetrator Race"]!=0)& (data["Perpetrator Sex"]!=0)]
newdata=newdata[newdata["Perpetrator Ethnicity"]!=0]

In [23]:
bins = [-1, 12, 18, 60, np.inf]
labels = ['child', 'teenager', 'adult', 'elder']
age_groups = pd.cut(newdata["Perpetrator Age"], bins, labels=labels)
newdata['age_group'] = age_groups

In [24]:
x=newdata[["State","Year","Month","Victim Count","Perpetrator Count",
           "Victim Sex","Victim Age","Victim Race","Victim Ethnicity",
          "Weapon","Relationship","Crime Type"]]
y=newdata[["age_group"]]

In [25]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=10)

1. Random Forest

In [26]:
clfage=RandomForestClassifier(n_estimators=100)
clfage.fit(x_train,y_train.values.ravel())
y_age_pred=clfage.predict(x_test)
print(metrics.accuracy_score(y_test,y_age_pred))

0.8136230672795653


2. Logistic Regression

In [27]:
loregage=LogisticRegression(solver='lbfgs', multi_class='auto',dual = False,max_iter=1000)
loregage.fit(x_train,y_train.values.ravel())
y_age_pred=loregage.predict(x_test)
print(metrics.accuracy_score(y_test,y_age_pred))

C:\Users\Kmar\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.8061272461345591
